In [1]:
import os, sys
sys.path.append('../../')
import Config
from src.Simulators.OpenRaveSimulator import *
from openravepy import *
from tf.transformations import *
from src.OpenraveUtils import *
from prpy.planning import sbpl

In [2]:
# Load environment
sim = OpenRaveSimulator(Config.OPENRAVE_ENV_XML)
env = sim.env
robot = env.GetRobot(Config.ROBOT_NAME)
# old = robot.GetActiveDOFIndices()
# Set affine dofs as active dofs
robot.SetActiveDOFs([],DOFAffine.X|DOFAffine.Y|DOFAffine.RotationAxis)

In [3]:
# Loading Planner paramns
import yaml
with open(Config.MISC_DIR+"RobotModels/fetch/URDF/sbpl_global_params.yaml", 'r') as stream:
    try:
        planner_params = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)
    stream.close()

In [4]:
# SBPL Planner
planner = sbpl.SBPLPlanner()
planner.setupEnv(env)
planner.SetPlannerParameters(planner_params)

In [5]:
# End Pose
pose = np.eye(4)
pose[0,3]=4.0
pose[1,3]=3.0
# Trajectory
traj = planner.PlanToBasePose(robot,pose)

In [6]:
# Smoothen the trajectory
_ = planningutils.SmoothAffineTrajectory(traj, [4.,4.,4.], [.5, .5, .5], plannername='LinearTrajectoryRetimer')

In [7]:
# Execute Trajectory
with robot:
    robot.GetController().SetPath(traj)
robot.WaitForController(0)

True